# To do list
1. generate a template first
2. add sections one by one manually
3. replace the current section code by functions later


In [183]:
# pre defien functions

def general_info(CreatedBy, LastModifiedBy, 
                 CreationDate, LastModifiedDate, ProductVersion, SchemaVersion,
                SavedBy, SaveDate, LastSavedBy, LastSaveDate):
    root = ET.Element("LibraryDesign", ID="0", ConCheck="0", LinkID="", PersistState="0")
    ET.SubElement(root, "Name", Null="1")
    ET.SubElement(root, "CreatedBy").text = CreatedBy
    ET.SubElement(root, "LastModifiedBy").text = LastModifiedBy
    ET.SubElement(root, "Project", Null="1")
    ET.SubElement(root, "Notebook", Null="1")
    ET.SubElement(root, "Pages", Null="1")
    ET.SubElement(root, "Comments", Null="1")
    ET.SubElement(root, "Staff", Null="1")
    ET.SubElement(root, "Keywords", Null="1")
    ET.SubElement(root, "Type", Null="1")
    ET.SubElement(root, "CreationDate").text = CreationDate
    ET.SubElement(root, "LastModificationDate").text = LastModifiedDate
    ET.SubElement(root, "ProductVersion").text = ProductVersion
    ET.SubElement(root, "SchemaVersion").text = SchemaVersion
    ET.SubElement(root, "Status").text = "0"
    ET.SubElement(root, "Flags").text = "0"
    ET.SubElement(root, "SavedBy").text = SavedBy
    ET.SubElement(root, "SaveDate").text = SaveDate
    ET.SubElement(root, "LastSavedBy").text = LastSavedBy
    ET.SubElement(root, "LastSaveDate").text = LastSaveDate

    ET.SubElement(root, "OriginID").text = OriginID
    ET.SubElement(root, "OriginDBID", Null="1")
    ET.SubElement(root, "LSTypes")
    return root


def generate_LibElem(Root, LinkID, Position, Plate):
    elem_i = ET.SubElement(Root, "LSLibraryElement", ID="0", ConCheck="0", PersistState="0", LinkID=LinkID)
    ET.SubElement(elem_i, "Position").text = Position
    ET.SubElement(elem_i, "Status").text = "0"
    ET.SubElement(elem_i, "Flags").text = "0"
    text = LibraryName + "("+Plate+")"
    ET.SubElement(elem_i, "Name").text = text

def generate_mapAmount(Root, Row, Column, Value):
    mapAmount = ET.SubElement(Root, "LSMapAmount")
    ET.SubElement(mapAmount, "Row").text=Row
    ET.SubElement(mapAmount, "Column").text=Column
    ET.SubElement(mapAmount, "Value").text=Value
    ET.SubElement(mapAmount, "MappedValue").text=Value

def add_map(Root, Index, MapName, MapType, Description, LibraryName, Tag, 
                 Selected, Chemical=None, Value=None, Discrete_Values=None):
    layer = ET.SubElement(Root, "LSLayer")
    ET.SubElement(layer, "Index").text = Index
    maps = ET.SubElement(layer, "LSMaps")
#     if MapName == "dispense":
#         sourceMap = ET.SubElement(maps, "LSSourceMap")
#     else:
#         sourceMap = ET.SubElement(maps, "LSParameterMap")
    
    if MapName == "dispense":
        sourceMap = ET.SubElement(maps, "LSSourceMap")
        ET.SubElement(sourceMap, "Unit", Name="ul").text="513"
        ET.SubElement(sourceMap, "MappedUnit", Name="ul").text="513"
    elif MapName == "stir":
        sourceMap = ET.SubElement(maps, "LSParameterMap")
        ET.SubElement(sourceMap, "Unit", Name="rpm").text="3585"
        ET.SubElement(sourceMap, "ParameterName").text="StirRate"
    elif MapName == "heat":
        sourceMap = ET.SubElement(maps, "LSParameterMap")
        ET.SubElement(sourceMap, "Unit", Name="degC").text="1281"
    else:
        print(f'Map name: {MapName} not found.')
    ET.SubElement(sourceMap, "MapType").text=MapType
    ET.SubElement(sourceMap, "Description").text=Description
    ET.SubElement(sourceMap, "Annotation", Null="1")
    ET.SubElement(sourceMap, "LibraryName").text=LibraryName
#     if MapName == "stir":
#         ET.SubElement(sourceMap, "ParameterName").text="StirRate"
    
    calparas = ET.SubElement(sourceMap, "LSCalcParameters")
    calpara = ET.SubElement(calparas, "LSCalcParameter", Name="Value")
    ET.SubElement(calpara, "Flags").text="0"
    ET.SubElement(calpara, "ID").text="1"
    ET.SubElement(calpara, "Value", Type="5", DI="0").text=Value

    mapTag = ET.SubElement(sourceMap, "Tags", Name="Tags")
    ET.SubElement(mapTag, "Tag").text=Tag
    mapAmounts = ET.SubElement(sourceMap, "LSMapAmounts")
    for i in Selected:
        generate_mapAmount(mapAmounts, str(i[0]), str(i[1]), Value)

def add_chemical(Root, Index, MapType, UnitName, UnitCode, Description, LibraryName, Chemical, Tag, 
                 Selected, Value=None, Discrete_Values=None):
    layer = ET.SubElement(Root, "LSLayer")
    ET.SubElement(layer, "Index").text = Index
    maps = ET.SubElement(layer, "LSMaps")
    sourceMap = ET.SubElement(maps, "LSSourceMap")
    ET.SubElement(sourceMap, "MapType").text=MapType
    ET.SubElement(sourceMap, "Unit", Name=UnitName).text=UnitCode
    ET.SubElement(sourceMap, "MappedUnit", Name=UnitName).text=UnitCode
    ET.SubElement(sourceMap, "Description").text=Description
    ET.SubElement(sourceMap, "Annotation", Null="1")
    ET.SubElement(sourceMap, "LibraryName").text=LibraryName

    mapSource = ET.SubElement(sourceMap, "Source")
    ET.SubElement(mapSource, "Name").text=Chemical
    ET.SubElement(mapSource, "SourceArrayName")
    ET.SubElement(mapSource, "SourcePos")

    calparas = ET.SubElement(sourceMap, "LSCalcParameters")
    if MapType =="Uniform":
        calpara = ET.SubElement(calparas, "LSCalcParameter", Name="Value")
        ET.SubElement(calpara, "Flags").text="0"
        ET.SubElement(calpara, "ID").text="1"
        ET.SubElement(calpara, "Value", Type="5", DI="0").text=Value

    mapTag = ET.SubElement(sourceMap, "Tags", Name="Tags")
    ET.SubElement(mapTag, "Tag").text=Tag
    mapAmounts = ET.SubElement(sourceMap, "LSMapAmounts")
    if MapType =="Uniform":
        for i in Selected:
            generate_mapAmount(mapAmounts, str(i[0]), str(i[1]), Value)
    if MapType == "Discrete":
        for j, i in enumerate(Selected):
            generate_mapAmount(mapAmounts, str(i[0]), str(i[1]), str(Discrete_Values[j]))

def add_stir(Root, Index, MapType, UnitName, UnitCode, Description, LibraryName, Value, Tag, Selected):
    layer = ET.SubElement(Root, "LSLayer")
    ET.SubElement(layer, "Index").text = Index
    maps = ET.SubElement(layer, "LSMaps")
    sourceMap = ET.SubElement(maps, "LSParameterMap")
    ET.SubElement(sourceMap, "MapType").text=MapType
    ET.SubElement(sourceMap, "Unit", Name=UnitName).text=UnitCode
    #ET.SubElement(sourceMap, "MappedUnit", Name=UnitName).text=UnitCode
    ET.SubElement(sourceMap, "Description").text=Description
    ET.SubElement(sourceMap, "Annotation", Null="1")
    ET.SubElement(sourceMap, "LibraryName").text=LibraryName
    ET.SubElement(sourceMap, "ParameterName").text="StirRate"

    calparas = ET.SubElement(sourceMap, "LSCalcParameters")
    calpara = ET.SubElement(calparas, "LSCalcParameter", Name="Value")
    ET.SubElement(calpara, "Flags").text="0"
    ET.SubElement(calpara, "ID").text="1"
    ET.SubElement(calpara, "Value", Type="5", DI="0").text=Value

    mapTag = ET.SubElement(sourceMap, "Tags", Name="Tags")
    ET.SubElement(mapTag, "Tag").text=Tag
    mapAmounts = ET.SubElement(sourceMap, "LSMapAmounts")
    for i in Selected:
        generate_mapAmount(mapAmounts, str(i[0]), str(i[1]), Value)
def add_heating(Root, Index, MapType, UnitName, UnitCode, Description, LibraryName, Value, 
                Tag, Selected, Value_min, Value_max, Step, Shape, Direction):
    layer = ET.SubElement(Root, "LSLayer")
    ET.SubElement(layer, "Index").text = Index
    maps = ET.SubElement(layer, "LSMaps")
    sourceMap = ET.SubElement(maps, "LSParameterMap")
    ET.SubElement(sourceMap, "MapType").text=MapType
    ET.SubElement(sourceMap, "Unit", Name=UnitName).text=UnitCode
    #ET.SubElement(sourceMap, "MappedUnit", Name=UnitName).text=UnitCode
    ET.SubElement(sourceMap, "Description").text=Description
    ET.SubElement(sourceMap, "Annotation", Null="1")
    ET.SubElement(sourceMap, "LibraryName").text=LibraryName
    ET.SubElement(sourceMap, "ParameterName").text="HeatingTemp"

    calparas = ET.SubElement(sourceMap, "LSCalcParameters")
    if MapType == "Uniform":
        calpara = ET.SubElement(calparas, "LSCalcParameter", Name="Value")
        ET.SubElement(calpara, "Flags").text="0"
        ET.SubElement(calpara, "ID").text="1"
        ET.SubElement(calpara, "Value", Type="5", DI="0").text=Value
    if MapType == "Gradient":
        calparamin = ET.SubElement(calparas, "LSCalcParameter", Name="Min Amount")
        ET.SubElement(calparamin, "Flags").text="0"
        ET.SubElement(calparamin, "ID").text="1"
        ET.SubElement(calparamin, "Value", Type="5", DI="0").text=str(Value_min)
        calparamax = ET.SubElement(calparas, "LSCalcParameter", Name="Max Amount")
        ET.SubElement(calparamax, "Flags").text="0"
        ET.SubElement(calparamax, "ID").text="2"
        ET.SubElement(calparamax, "Value", Type="5", DI="0").text=str(Value_max)
        calparastep = ET.SubElement(calparas, "LSCalcParameter", Name="Step")
        ET.SubElement(calparastep, "Flags").text="2"
        ET.SubElement(calparastep, "ID").text="3"
        ET.SubElement(calparastep, "Value", Type="3", DI="1").text=str(Step)
        # shape
        calparasp = ET.SubElement(calparas, "LSCalcParameter", Name="Shape")
        ET.SubElement(calparasp, "Flags").text="2"
        ET.SubElement(calparasp, "ID").text="4"
        ET.SubElement(calparasp, "Value", Type="5", DI="0.5").text=str(Shape)
        # direction 
        calparadir = ET.SubElement(calparas, "LSCalcParameter", Name="Direction")
        ET.SubElement(calparadir, "Flags").text="1"
        ET.SubElement(calparadir, "ID").text="5"
        ET.SubElement(calparadir, "Value", Type="3", DI="0").text=str(Direction)
        
    mapTag = ET.SubElement(sourceMap, "Tags", Name="Tags")
    ET.SubElement(mapTag, "Tag").text=Tag
    
    mapAmounts = ET.SubElement(sourceMap, "LSMapAmounts")
    if MapType == "Uniform":
        for i in Selected:
            generate_mapAmount(mapAmounts, str(i[0]), str(i[1]), Value)
    if MapType =="Gradient":
        selected_count = len(Selected)
        increment = (Value_max - Value_min)/(selected_count/Step-1)
        current_value = Value_min
        for count, i in enumerate(Selected):
            generate_mapAmount(mapAmounts, str(i[0]), str(i[1]), str(current_value))
            if (count+1)%Step == 0:
                current_value += increment
                

def add_LibView(Root, TopPos, LeftPos, Width, Height):
    libview = ET.SubElement(Root, "LSLibraryView")
    ET.SubElement(libview, "LibraryUID").text="1"
    ET.SubElement(libview, "ActiveLayer").text="1"
    ET.SubElement(libview, "TopWindowPosition").text=TopPos
    ET.SubElement(libview, "LeftWindowPosition").text=LeftPos
    ET.SubElement(libview, "WindowWidth").text=Width
    ET.SubElement(libview, "WindowHeight").text=Height
    ET.SubElement(libview, "InvisibleSource")
    
def generate_chemical(Root, chemName, chemColor, density, molW, chemUnit, dispenseUnit, AmountDispensed):
    chem = ET.SubElement(Root, "LSChemical", ID="0", ConCheck="0", PersistState="0", LinkID="")
    ET.SubElement(chem, "Version").text = chemVersion
    ET.SubElement(chem, "Name").text = chemName
    ET.SubElement(chem, "Color").text = chemColor
    ET.SubElement(chem, "UnitDispensed", Name=chemUnit).text = dispenseUnit
    ET.SubElement(chem, "AmountDispensed").text = AmountDispensed
    ET.SubElement(chem, "Density").text = density
    ET.SubElement(chem, "MolW").text = molW
    ET.SubElement(chem, "SourceArrayName", Null="1")
    ET.SubElement(chem, "SourceArrayID").text = "0"
    ET.SubElement(chem, "Position").text = "1"
    ET.SubElement(chem, "WSAmount").text = AmountDispensed
    ET.SubElement(chem, "WSUnit", Name=chemUnit).text = dispenseUnit
    ET.SubElement(chem, "WSBarcode", Null="1")
    ET.SubElement(chem, "SourceArrayDBID", Null="1")
    ET.SubElement(chem, "LSEquivalences")
    ET.SubElement(chem, "NamedAttributes")

def generate_parameter(Root, Name):
    para = ET.SubElement(Root, "LSParameter", ID="0", ConCheck="0", PersistState="0", LinkID="")
    ET.SubElement(para, "Version").text = "2"
    ET.SubElement(para, "Name").text = Name
    if Name == "StirRate":
        Type = "Rate"
        DefaultUnit ="2049"
    elif Name == "HeatingTemp":
        Type = "Temperature"
        DefaultUnit ="1281"
    elif Name == "SetTimer":
        Type = "Time"
        DefaultUnit ="769"
    else:
        print(f"Parameter name: {Name} not found.")
    ET.SubElement(para, "Type").text = Type
    ET.SubElement(para, "Description", Null="1")
    ET.SubElement(para, "CanVaryAcrossRows").text = "-1"
    ET.SubElement(para, "CanVaryAcrossCols").text = "-1"
    ET.SubElement(para, "DefaultUnit").text = DefaultUnit
    ET.SubElement(para, "SourceUnit").text = "513"
    ET.SubElement(para, "Expression", Null="1")
    ET.SubElement(para, "DecimalPlaces").text = "2"
def generate_parameter_StirRate(Root):
    para = ET.SubElement(Root, "LSParameter", ID="0", ConCheck="0", PersistState="0", LinkID="")
    ET.SubElement(para, "Version").text = "2"
    ET.SubElement(para, "Name").text = "StirRate"
    ET.SubElement(para, "Type").text = "Rate"
    ET.SubElement(para, "Description", Null="1")
    ET.SubElement(para, "CanVaryAcrossRows").text = "-1"
    ET.SubElement(para, "CanVaryAcrossCols").text = "-1"
    ET.SubElement(para, "DefaultUnit").text = "2049"
    ET.SubElement(para, "SourceUnit").text = "513"
    ET.SubElement(para, "Expression", Null="1")
    ET.SubElement(para, "DecimalPlaces").text = "2"
def generate_parameter_HeatingTemp(Root):
    para = ET.SubElement(Root, "LSParameter", ID="0", ConCheck="0", PersistState="0", LinkID="")
    ET.SubElement(para, "Version").text = "2"
    ET.SubElement(para, "Name").text = "HeatingTemp"
    ET.SubElement(para, "Type").text = "Temperature"
    ET.SubElement(para, "Description", Null="1")
    ET.SubElement(para, "CanVaryAcrossRows").text = "-1"
    ET.SubElement(para, "CanVaryAcrossCols").text = "-1"
    ET.SubElement(para, "DefaultUnit").text = "1281"
    ET.SubElement(para, "SourceUnit").text = "513"
    ET.SubElement(para, "Expression", Null="1")
    ET.SubElement(para, "DecimalPlaces").text = "2"
def generate_parameter_SetTimer(Root):
    para = ET.SubElement(Root, "LSParameter", ID="0", ConCheck="0", PersistState="0", LinkID="")
    ET.SubElement(para, "Version").text = "2"
    ET.SubElement(para, "Name").text = "SetTimer"
    ET.SubElement(para, "Type").text = "Time"
    ET.SubElement(para, "Description", Null="1")
    ET.SubElement(para, "CanVaryAcrossRows").text = "-1"
    ET.SubElement(para, "CanVaryAcrossCols").text = "-1"
    ET.SubElement(para, "DefaultUnit").text = "769"
    ET.SubElement(para, "SourceUnit").text = "513"
    ET.SubElement(para, "Expression", Null="1")
    ET.SubElement(para, "DecimalPlaces").text = "2"

def generate_library(LibraryName, LibDescription, LibRow, LibColumn):
    libdoc = ET.SubElement(root, "LSLibraries2")
    lib = ET.SubElement(libdoc, "LSLibrary", ID="0", ConCheck="0", PersistState="0", LinkID="")
    ET.SubElement(lib, "Version").text = "2"
    ET.SubElement(lib, "Index").text = "1"
    ET.SubElement(lib, "LibraryID").text = "0"
    ET.SubElement(lib, "Name").text = LibraryName
    ET.SubElement(lib, "Description").text = LibDescription
    ET.SubElement(lib, "Rows").text = LibRow
    ET.SubElement(lib, "Columns").text = LibColumn
    ET.SubElement(lib, "Shape").text = "0"
    ET.SubElement(lib, "Flags").text = "0"
    ET.SubElement(lib, "Color").text = "65280"
    ET.SubElement(lib, "Notebook", Null="1")
    ET.SubElement(lib, "Pages", Null="1")
    ET.SubElement(lib, "Project", Null="1")
    ET.SubElement(lib, "SynID").text = "0"
    ET.SubElement(lib, "SynConCheck").text = "0"
    ET.SubElement(lib, "IsSourceLibrary").text = "0"
    ET.SubElement(lib, "Staff", Null="1")
    ET.SubElement(lib, "OriginLibDBID", Null="1")
    ET.SubElement(lib, "OriginLibID").text = "0"

    lssm = ET.SubElement(lib, "LSSourceMgr")
    ET.SubElement(lssm, "LSTypes")
    ET.SubElement(lssm, "LSChemicals")
    ET.SubElement(lssm, "LSMixtures")
    ET.SubElement(lssm, "LSParameters")

    ET.SubElement(lib, "NamedAttributes")
    # Library elements
    libelem = ET.SubElement(lib, "LSLibraryElements")
    row = int(LibRow)
    col = int(LibColumn)
    row_letter_list = ["A", "B", "C", "D"]
    for i in range(row):
        for j in range(col):
            symbol = row_letter_list[i]+str(j+1)
            generate_LibElem(libelem, str(col*i+j+1), str(col*i+j+1), symbol)
    return lib

In [184]:
import xml.etree.ElementTree as ET

################################## general information #############################
ProductVersion = "8.5.0.0"
SchemaVersion = "3.0"
OriginID = "0"
CreatedBy = "hermanuser"
CreationDate = "2022/08/02 16:07:52"
LastModifiedBy = "hermanuser"
LastModifiedDate = "2022/08/02 16:07:52"
SavedBy = "hermanuser"
SaveDate = "2022/08/02 16:07:52"
LastSavedBy = "hermanuser"
LastSaveDate = "2022/08/02 16:07:52"

# adding general doc info
root = general_info(CreatedBy, LastModifiedBy, 
                 CreationDate, LastModifiedDate, ProductVersion, SchemaVersion,
                SavedBy, SaveDate, LastSavedBy, LastSaveDate)

################################# chemicals ##################################
### adding chemicals
# chemicals (TODO: save as json)
chemVersion = "2"
# chemical 1
chemName1 = "toluene"
density1 = "0.867"
molW1 = "92.140000000000001"
chemColor1 = "255" # TODO: make a color dictionary/JSON file for color names and code
chemUnit1 = "ul"  #TODO: make a "unit : code" dictionary/JSON file
dispenseUnit1 = "513"
AmountDispensed1 = "800." # TODO: calculate automatically

#chemical 2
chemName2 = "water"
density2 = "1"
molW2 = "18."
chemColor2 = "16711680"
chemUnit2 = "ul"
dispenseUnit2 = "513"
AmountDispensed2 = "450."

#chemical 3
chemName3 = "oleic acid"
density3 = "0.895"
molW3 = "282.46"
chemColor3 = "12640511"
chemUnit3 = "ml"
dispenseUnit3 = "514"
AmountDispensed3 = "0."

chemdoc = ET.SubElement(root, "LSChemicals")
generate_chemical(chemdoc, chemName1, chemColor1, density1, molW1, chemUnit1, dispenseUnit1, AmountDispensed1)
generate_chemical(chemdoc, chemName2, chemColor2, density2, molW2, chemUnit2, dispenseUnit2, AmountDispensed2)
generate_chemical(chemdoc, chemName3, chemColor3, density3, molW3, chemUnit3, dispenseUnit3, AmountDispensed3)

################################### mixtures ####################################
mixdoc = ET.SubElement(root, "LSMixtures")

################################### parameters ##################################
paradoc = ET.SubElement(root, "LSParameters")
generate_parameter_StirRate(paradoc) # alternatively, use "generate_parameter(Root, Name)""
generate_parameter_HeatingTemp(paradoc)
generate_parameter_SetTimer(paradoc)

################################### libraries #####################################
LibraryName1 = "Xiaojing's reaction plate"
LibColumn1 = "4"
LibRow1 = "2"
LibDescription1 = "library_description_text_box" 
lib = generate_library(LibraryName1, LibDescription1, LibRow1, LibColumn1)

################################### Layers ########################################
layerdoc = ET.SubElement(root, "LSLayers")
Index ="1"
MapType = "Uniform"
UnitName = "ul"
UnitCode = dispenseUnit
Description = "Add 100 ul toluene to Xiaojing's reaction plate (A1:B4) in ul"
LibraryName = "Xiaojing's reaction plate"
Chemical = "toluene"
Value = "100"
Tag = "1000uLTip"
selected_wells = [[1,1],[1,2],[1,3],[1,4],[2,1],[2,2],[2,3],[2,4]] 
add_chemical(layerdoc, Index, MapType, UnitName, UnitCode, Description, LibraryName, Chemical, Tag, selected_wells, Value)   

# layer 2 
Index ="2"
MapType = "Uniform"
UnitName = "rpm"
UnitCode = "3585"
Description = "Set StirRate to 500 rpm; Xiaojing's reaction plate (A1:B4)"
LibraryName = "Xiaojing's reaction plate"
Value = "500."
Tag = "Herman"
selected_wells = [[1,1],[1,2],[1,3],[1,4],[2,1],[2,2],[2,3],[2,4]] 
#add_stir(layerdoc, Index, MapType, UnitName, UnitCode, Description, LibraryName, Value, Tag, selected_wells)
add_map(layerdoc, Index, "stir", MapType, Description, LibraryName, Tag, 
                 selected_wells, Chemical=None, Value=Value, Discrete_Values=None)
# layer 3
Index ="3"
MapType = "Gradient"
UnitName = "degC"
UnitCode = "1281"
Description = "Set HeatingTemp as a linear gradient (left to right, step 1) 100 to 200 in degC; Xiaojing's reaction plate (B1:B4)"
LibraryName = "Xiaojing's reaction plate"
Value = "500."
Tag = "Herman"
selected_wells = [[2,1],[2,2],[2,3],[2,4]] 
add_heating(layerdoc, Index, MapType, UnitName, UnitCode, Description, LibraryName, Value, Tag, selected_wells, 
        Value_min=100, Value_max=200, Step=1, Shape=0, Direction=0)

# layer 4
Index ="4"
MapType = "Discrete"
UnitName = "ul"
UnitCode = dispenseUnit
Description = "Add discrete amounts of water in ul to Xiaojing's reaction plate (B1:B4) in ul"
LibraryName = "Xiaojing's reaction plate"
Chemical = "water"
Tag = "1000uLTip"
selected_wells = [[2,1],[2,2],[2,3],[2,4]] 
discrete_values = [100,50,100,200]
add_chemical(layerdoc, Index, MapType, UnitName, UnitCode, Description, LibraryName, Chemical, Tag, selected_wells, Discrete_Values=discrete_values)   

              



# # ViewMgrData
# vmddoc = ET.SubElement(root, "ViewMgrData")
# vm = ET.SubElement(vmddoc, "LSViewMgr")
# ET.SubElement(vm, "ActiveLayer").text="1"
# ET.SubElement(vm, "ActiveMap").text="1"
# ET.SubElement(vm, "SelectionMode").text="0"
# ET.SubElement(vm, "ChartType").text="0"
# ET.SubElement(vm, "ActivePlate").text="Xiaojing's reaction plate"
# ET.SubElement(vm, "CompositionType").text="0"
# ET.SubElement(vm, "PercentType").text="3"
# ET.SubElement(vm, "ShowWindowState").text="0"
# ET.SubElement(vm, "SizeByWhat").text="Total"
# ET.SubElement(vm, "ScaleByDesign").text="0"
# ET.SubElement(vm, "ShapeFactor").text="0."
# ET.SubElement(vm, "StructureType").text="All"
# ET.SubElement(vm, "StructureColumns").text="0"
# ET.SubElement(vm, "TabIndex").text="3"
# ET.SubElement(vm, "SplitterHoriz").text="0.67"
# ET.SubElement(vm, "SplitterVert").text="0.2"
# ET.SubElement(vm, "SplitterFavor").text="0.2"
# ET.SubElement(vm, "InvisibleSource")
# collection = ET.SubElement(vm, "LibraryViewCollections")
# libviews = ET.SubElement(collection, "LSLibraryViews")
# ET.SubElement(libviews, "LibraryUID").text="1"

# add_LibView(libviews, "10", "16", "161", "115")
# add_LibView(libviews, "0", "0", "300", "200")

# fo = ET.SubElement(vm, "FieldOrganizer")
# ET.SubElement(fo, "Fields")


# cumdoc = ET.SubElement(root, "CustomData")
# ET.SubElement(cumdoc, "CustomDataColl")



tree = ET.ElementTree(root)
tree.write("test.xml")